In [34]:
import numpy as np
import cv2

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
wc = 4  ## 체스 보드 가로 패턴 개수 - 1
hc = 6  ## 체스 보드 세로 패턴 개수 - 1
objp = np.zeros((wc * hc, 3), np.float32)
objp[:, :2] = np.mgrid[0:wc, 0:hc].T.reshape(-1, 2)

objpoints = []
imgpoints = []

file = "./Input2/15cm.png"  ## 체스 보드 이미지
input_name = "002.png"
dist_file = "./Input2/"+input_name  ## 왜곡된 이미지 (같은 화각으로 찍은 이미지)

img = cv2.imread(file)
#_img = img
_img = cv2.resize(img, dsize=(456,608), interpolation = cv2.INTER_AREA)
# cv2.imshow('img', _img)
gray = cv2.cvtColor(_img, cv2.COLOR_BGR2GRAY) ## gray scale로 바꾸기
# cv2.waitKey(0)

ret, corners = cv2.findChessboardCorners(gray, (wc, hc), None)  ## 체스 보드 찾기
print(ret) 
## 만약 ret값이 False라면, 체스 보드 이미지의 패턴 개수를 맞게 했는지 확인하거나 (wc, hc) 체스 보드가 깔끔하게 나온 이미지를 가져와야 한다

if ret == True:
    objpoints.append(objp)

    corners2 = cv2.cornerSubPix(gray, corners, (10, 10), (-1, -1), criteria) ## Canny86 알고리즘으로 도형이 겹치는 코너 점을 찾는다
    imgpoints.append(corners2)

    ## 찾은 코너 점들을 이용해 체스 보드 이미지에 그려넣는다
    img = cv2.drawChessboardCorners(_img, (wc, hc), corners2, ret)
    # cv2.imshow('img', img)

    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)  ## 왜곡 펴기
    h, w = img.shape[:2]
    newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1)
    ## mtx = getOptimalNewCameraMatrix parameter alpha 
    ## dist = Free scaling parameter 
    ## 4번째 인자 = between 0 (when all the pixels in the undistorted image are valid) and 1 (when all the source image pixels are retained in the undistorted image)
    ## 1에 가까울수록 왜곡을 펼 때 잘라낸 부분들을 더 보여준다
    ## 전체를 보고 싶다면 1, 펴진 부분만 보고 싶다면 0에 가깝게 인자 값을 주면 된다
    #dst = cv2.undistort(img, mtx, dist) ## getOptimalNewCameraMatrix 함수를 쓰지 않은 이미지
    #dst2 = cv2.undistort(img, mtx, dist, None, newcameramtx) ## 함수를 쓴 이미지
    #cv2.imshow('num0', img)
    #cv2.imshow('num1', dst)
    #cv2.imshow('num2', dst2)
    #cv2.waitKey(0)
    
    dst = cv2.undistort(img, mtx, dist)
    cv2.imwrite("./Output/checkboard.png", dst)
    
    dist_img = cv2.imread(dist_file)
    dst = cv2.undistort(dist_img, mtx, dist)
    cv2.imwrite("./Output/"+input_name, dst)

#cv2.destroyAllWindows()

True


In [78]:
import numpy as np
import glob, cv2

# 종료 기준(termination criteria)를 정한다.
criteria = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30 ,0.001)
wc = 8  ## 체스 보드 가로 패턴 개수 - 1
hc = 6  ## 체스 보드 세로 패턴 개수 - 1
# Object Point(3D)를 준비한다. (0,0,0),(1,0,0),(2,0,0)... 처럼
objp = np.zeros((wc * hc, 3), np.float32)
objp[:, :2] = np.mgrid[0:wc, 0:hc].T.reshape(-1, 2)

# 이미지로 부터의 Object point와 Image points를 저장하기 위한 배열
objpoints = [] # 실제 세계의 3D 점들 
imgpoints = [] # 2D 이미지의 점들

# 전체 path를 받기 위해 os말고 glob 사용
images = glob.glob('./Base3/*.jpg')

for name in images:
    img = cv2.imread(name)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 체스판의 코너들 찾기
    ret, corners = cv2.findChessboardCorners(gray,(wc, hc),None)
    print(name, ret)
    # 찾았으면, Object points, Image points 추가하기 (이후에 수정한다)
    if ret == True:
        objpoints.append(objp)

        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners2)

        # 코너를 그리고 봐보자
        img = cv2.drawChessboardCorners(img,(wc,hc),corners2,ret)
        #cv2.imshow('img',img)
        #cv2.waitKey(2000)
cv2.destroyAllWindows()

./Base3\Check01.jpg True
./Base3\Check02.jpg True
./Base3\Check03.jpg True
./Base3\Check04.jpg True
./Base3\Check05.jpg True
./Base3\Check06.jpg True
./Base3\Check07.jpg True
./Base3\Check08.jpg True
./Base3\Check09.jpg True
./Base3\Check10.jpg True
./Base3\Check11.jpg True
./Base3\Check12.jpg True


In [74]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

In [76]:
import os
path = "./Input2/"
file_list = os.listdir(path)
Image_List = []
for i in range(len(file_list)):
    Num = len(file_list[i])
    if Num > 4 and file_list[i][-4:Num] == ".png":
        Image_List.append(file_list[i])
        
for input_image in Image_List:
    dist_img = cv2.imread("./Input2/"+input_image)
    dst = cv2.undistort(dist_img, mtx, dist)
    cv2.imwrite("./Output/"+input_image, dst)
    print("./Output/"+input_image+" is Saved.")

./Output/Silhouette_square2.png is Saved.
./Output/Silhouette_square3.png is Saved.
